#### Import Modules

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import fiona
import folium
import shapefile
import rasterstats
import rasterio
from folium.plugins import MarkerCluster
from folium.features import ColorLine
from folium.features import GeoJson
import json
import pdb
import copy

In [2]:
UW_coords = [47.655548, -122.303200]

In [3]:
routes_shp = gpd.read_file('../local_files/six_routes.shp')
routes_list = routes_shp['ROUTE_NUM'].tolist()
routes_list

[75, 48, 45, 7, 40, 50]

In [4]:
route_45 = routes_shp[routes_shp['ROUTE_NUM'] == 45]
route_45

,SHAPE_Leng,ROUTE_NUM,Id,OBJECTID,geometry
2,0.0,45,0,0,LINESTRING Z (-122.3042444001078 47.6504264662...


#### Extract elevation from lidar and compute gradient between the two point

In [22]:
elevation_45 = rasterstats.point_query(route_45, '../local_files/sea_north.tif')
elevation_gradient = np.abs(np.diff(elevation_45[0]))
# elevation_gradient

#### Extract the coordinates from the shape file

In [23]:
from shapely.geometry import mapping
route_geometry = route_45.geometry.values # list of shapely geometries
route_geometry = [mapping(route_geometry[0])]
coordinates_route = route_geometry[0]['coordinates']
linestring_route = []
for i in range(len(coordinates_route)):
    linestring_route.append(coordinates_route[i][:2])
linestring_route_df = pd.DataFrame()  
linestring_route_df['coordinates'] = linestring_route

#### Functions 

In [24]:
from shapely.geometry import LineString
def make_lines(gdf, gradient, idx, geometry = 'geometry'):   
    coordinate_1 = gdf.loc[idx]['coordinates']
    coordinate_2 = gdf.loc[idx + 1]['coordinates']
    line = LineString([coordinate_1, coordinate_2])
    
    data = {'gradient': gradient,
            'geometry':[line]}
    df_line = pd.DataFrame(data, columns = ['gradient', 'geometry'])
    
    return df_line

In [25]:
df_route = pd.DataFrame(columns = ['gradient', 'geometry'])
for idx in range(len(linestring_route_df) - 1):
    df_linestring = make_lines(linestring_route_df, elevation_gradient[idx], idx)
    df_route = pd.concat([df_route, df_linestring])


#### Convert dataframe with line routes above to geodataframe, then to json and finally save json file.

In [26]:
gdf_route = gpd.GeoDataFrame(df_route)
route_json = gdf_route.to_json()
# with open('Route_45.json', 'w') as json_file:
#     json.dump(route_json, json_file)

#### Display route on map

In [31]:
import branca.colormap as cm
import math
# PuRd_09
# RdBu_04
figure_size = folium.Figure(height = 400)
route_map = folium.Map(location = UW_coords, zoom_start = 12)

max_elev = max(elevation_gradient)
min_elev = min(elevation_gradient)
linear_map = cm.linear.Paired_06.scale(min_elev, max_elev )
route_layer = folium.GeoJson(route_json, style_function = lambda feature: {
                             'color': linear_map(feature['properties']['gradient']),
                             'weight': 7})

route_layer.add_child
route_map.add_child(linear_map)
route_map.add_child(route_layer)
route_map.add_to(figure_size)